In [1]:
from langchain_community.utilities import SQLDatabase
from pyprojroot import here
import warnings
warnings.filterwarnings("ignore")

In [2]:
from dotenv import load_dotenv
import os

print("Environment variables are loaded:", load_dotenv())

Environment variables are loaded: True


In [3]:
# Load the LLM
from langchain.chat_models import AzureChatOpenAI
import os

model_name = os.getenv("AZURE_OPENAI_API_DEPLOYMENT_NAME")
azure_openai_api_key = os.environ["AZURE_OPENAI_API_KEY"]
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]

llm = AzureChatOpenAI(
    openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_deployment=model_name,
    model_name=model_name,
    temperature=0.0)

c:\Nikhil\CustomerChatBot\env\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [4]:
db_user = "root"
db_password = "1234"
db_host = "localhost"
db_name = "chatbot"

db = SQLDatabase.from_uri(f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}")

In [5]:
print(db.dialect)
print(db.get_usable_table_names())
print(db.table_info)

mysql
['customer', 'order', 'product', 'shipment']

CREATE TABLE `order` (
	order_id INTEGER NOT NULL, 
	customer_id INTEGER, 
	product_id INTEGER, 
	order_date DATE, 
	status VARCHAR(255), 
	quantity INTEGER, 
	total_price DECIMAL(10, 2), 
	PRIMARY KEY (order_id), 
	CONSTRAINT order_ibfk_1 FOREIGN KEY(customer_id) REFERENCES customer (customer_id), 
	CONSTRAINT order_ibfk_2 FOREIGN KEY(product_id) REFERENCES product (product_id)
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci

/*
3 rows from order table:
order_id	customer_id	product_id	order_date	status	quantity	total_price
300	100	200	2024-06-01	Shipped	1	999.99
301	101	201	2024-06-02	Processing	2	1599.98
302	102	202	2024-06-03	Delivered	1	199.99
*/


CREATE TABLE customer (
	customer_id INTEGER NOT NULL, 
	first_name VARCHAR(255), 
	last_name VARCHAR(255), 
	email VARCHAR(255), 
	phone_number VARCHAR(255), 
	address VARCHAR(255), 
	PRIMARY KEY (customer_id)
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_090

In [5]:
db

In [6]:
from langchain_core.prompts import ChatPromptTemplate

template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)


In [7]:
def get_schema(_):
    return db.get_table_info()

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough



sql_chain = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)


In [9]:
sql_chain.invoke({'question':"How many customer are there?"})

'SELECT COUNT(*) FROM customer;'

New Template

In [10]:
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)


In [11]:
def run_query(query): # query will be string
    return db.run(query)

In [12]:
run_query('select * from customer')

"[(100, 'John', 'Doe', 'john.doe@example.com', '123-456-7890', '123 Elm St.'), (101, 'Jane', 'Smith', 'jane.smith@example.com', '234-567-8901', '234 Oak St.'), (102, 'Alice', 'Johnson', 'alice.johnson@example.com', '345-678-9012', '345 Pine St.'), (103, 'Bob', 'Williams', 'bob.williams@example.com', '456-789-0123', '456 Maple St.'), (104, 'Carol', 'Brown', 'carol.brown@example.com', '567-890-1234', '567 Cedar St.'), (105, 'David', 'Jones', 'david.jones@example.com', '678-901-2345', '678 Birch St.'), (106, 'Eve', 'Garcia', 'eve.garcia@example.com', '789-012-3456', '789 Walnut St.'), (107, 'Frank', 'Miller', 'frank.miller@example.com', '890-123-4567', '890 Chestnut St.'), (108, 'Grace', 'Davis', 'grace.davis@example.com', '901-234-5678', '901 Redwood St.'), (109, 'Hank', 'Martinez', 'hank.martinez@example.com', '012-345-6789', '123 Sequoia St.')]"

Create Chain

In [13]:
full_chain = (
    RunnablePassthrough.assign(query=sql_chain).assign(
        schema=get_schema,
        response=lambda vars: run_query(vars["query"]),
    )
    | prompt_response
    | llm
)


In [14]:

full_chain.invoke({"question": "how many customer are there?"})



AIMessage(content='There are 10 customers in the database.', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 798, 'total_tokens': 807}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c5c93979-2935-4b4b-b908-52354b58d26f-0')